In [ ]:
import cv2
import numpy as np
import math


bg = None

hand_cascade = cv2.CascadeClassifier('training/data/hand_cascade_stage16.xml')

In [ ]:
#--------------------------------------------------
# To find the running average over the background
#--------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [ ]:

cap = cv2.VideoCapture(0)
num_frames = 0
aWeight = 0.5
while(cap.isOpened()):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    if num_frames < 30:
        run_avg(gray, aWeight)
    else:
        diff = cv2.absdiff(bg.astype("uint8"), gray)
        hands = hand_cascade.detectMultiScale(gray,6,6)
        for (x,y,w,h) in hands:
            cv2.rectangle(gray, (x-50, y-50), (x+w+50, y+h+50), (0, 0, 255), 0)
        cv2.imshow('diff', diff)
  
    
    cv2.imshow('gray',gray)
    num_frames += 1
    k = cv2.waitKey(10)
    if k == 27:
        cap.release()
        break
        
cv2.destroyAllWindows()
